In [ ]:
# Install necessary library
%pip install -U geemap

In [ ]:
# Install necessary library
!pip install leafmap

In [ ]:
# Import necessary libraries
import ee
import geemap
import leafmap
import datetime

In [ ]:
# To initialise the project
ee.Authenticate()
ee.Initialize(project='ee-rice-yield-prediction')

# Create an interactive map
Map = geemap.Map(basemap='SATELLITE')

In [ ]:
# Load the GAUL dataset (Second-Level Administrative Units)
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Filter Sabak Bernam administrative unit
sabak = gaul.filter(ee.Filter.eq("ADM2_NAME", "Sabak Bernam"))

# Draw the boundary of Sabak Bernam
sabak_boundary = ee.Image().byte().paint(
  featureCollection = sabak,
  width=3
)

# Specify the visualisation of the boundary of Sabak Bernam
sabak_vis = {
    "palette": "black"
}

# Add Sabak Bernam's boundary to the map as a layer
Map.addLayer(sabak_boundary, sabak_vis, "Sabak Bernam")

# Center the map on Sabak Bernam
Map.centerObject(sabak, 8)

# Display the map
Map

Map(center=[3.679628002569312, 101.09087962782114], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Function to mask the cloud in the satellite images
def maskClouds(image):
  qa = image.select('QA60')

  # Creating bit masks for clouds and cirrus clouds
  # Bits 10 and 11 represents clouds and cirrus clouds respectively
  # Shifting the bit 1 to the left by 10 positions for clouds bit mask and by 11 positions for cirrus clouds bit mask
  cloudMask = 1 << 10
  cirrusMask = 1 << 11

  # Create a mask where clouds and cirrus clouds will be masked out
  mask = qa.bitwiseAnd(cloudMask).eq(0).And(qa.bitwiseAnd(cirrusMask).eq(0))

  # Return the input image without clouds and cirrus clouds
  return image.updateMask(mask)

In [ ]:
# Function to add NDVI band to the input image
def ndvi_band(image):
    # Calculate the NDVI value using NIR (B8) band and Red (B4) band
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')

    # Add the NDVI band to the image and return the image
    return image.addBands(ndvi)

In [ ]:
# Load the Sentinel-2 image in Sabak Bernam area
sentinel = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterDate('2023-03-05', '2023-05-25')
              .filterBounds(sabak)
              .map(maskClouds))

# Define the visualisation parameters
sentinel_vis = {
    'min': 0.0,
    'max': 2000,
    'bands': ['B4', 'B3', 'B2'],
}

image=sentinel.median()

# Adding the image to the map as a layer
Map.addLayer(image, sentinel_vis, 'Sentinel-2')

# Get the NDVI value
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')

#Specify the visualisation for NDVI
ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green'],
}

# Adding the NDVI to the map as a layer
Map.addLayer(ndvi.clip(sabak), ndvi_vis, 'NDVI')

# Display the map
Map

Map(bottom=32398.0, center=[3.6778915094650726, 101.09383338149844], controls=(WidgetControl(options=['positio…

In [ ]:
# Obtain an image of Sabak Area
feature = sabak.first()

# Get the geometry of Sabak Bernam area
sabak_geometry = feature.geometry()

# Convert the geometry to a GeoJSON format
geometry_geojson = sabak_geometry.getInfo()

# Create a LeafMap
m = leafmap.Map(center=[3.5, 101.15587962782114], zoom=10, height="800px")

# Add the GeoJSON layer to the map using 'add_geojson'
m.add_geojson(geometry_geojson, layer_name="Boundary")

# Add satellite as basemap
m.add_basemap("SATELLITE")

# Display the map
m

Map(center=[3.5, 101.15587962782114], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [ ]:
#bbox = m.user_roi_bounds()
# Sekinchan area
bbox=[101.0821, 3.5211, 101.1456, 3.5777]
print("Sekinchan Area:", bbox)

Sekinchan Area: [101.0821, 3.5211, 101.1456, 3.5777]


In [ ]:
# Function to calculate NDVI value
def calculate_ndvi(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]),
        scale=10,
        maxPixels=1e10
    )

    # Set NDVI to an empty value if reduceRegion does not have output.
    ndvi = ee.List([stats.get('ndvi'), ]).reduce(ee.Reducer.firstNonNull())
    # Create a feature with null geometry
    # Set NDVI value and date as properties
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    feature = ee.Feature(None, {'ndvi': ndvi, 'date': date})
    return feature

# Define start and end dates for the first quarter
start_date = '2019-03-05'
end_date = '2019-05-25'

# Initialise values
year = 1
quarter = 1

# Define the start and end dates for the second quarter
second_quarter_start_date = '2019-09-05'
second_quarter_end_date = '2019-11-25'

while (quarter <= 2):
    while (year <= 3):
        # Create an image collection of Sabak Bernam
        collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                .filterDate(start_date, end_date)
                .filterBounds(sabak)
                .map(maskClouds)
                .map(ndvi_band))

        # Filter Sekinchan area
        sekinchan = (collection.select('ndvi')
            .filterBounds(ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]))
        )

        # Get NDVI values for each image in the collection
        time_series = sekinchan.map(calculate_ndvi)

        # Define a name for the exported CSV file
        if (quarter==1):
          filename = "NDVI_" + start_date[0:4] + "_Q1"
        elif (quarter==2):
          filename = "NDVI_" + start_date[0:4] + "_Q2"

        # Export the time series to a CSV file
        task = ee.batch.Export.table.toDrive(
            collection=time_series,
            description='ndvi',
            folder='ndvi',
            fileNamePrefix=filename,
            fileFormat='CSV'
        )

        # Start the export task
        task.start()

        # Convert the start and end dates to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

        # Increment the year by 1
        start_date_dt = start_date_dt.replace(year=start_date_dt.year + 1)
        end_date_dt = end_date_dt.replace(year=end_date_dt.year + 1)

        # Convert the datetime objects back to strings
        start_date = start_date_dt.strftime('%Y-%m-%d')
        end_date = end_date_dt.strftime('%Y-%m-%d')

        year = year + 1

    quarter = quarter + 1

    # Set the start and end dates for the second quarter
    start_date = second_quarter_start_date
    end_date = second_quarter_end_date

    year = 1

In [ ]:
# Check task status
task.status()